In [6]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from DeepPurpose.dataset import *

## 1. load data by deeppurpose
- we load the "AID1706_SARS_CoV_3CL" dataset by the DeepPurpose package
  - The dataset could be found at https://pubchem.ncbi.nlm.nih.gov/bioassay/1706
- and select the first 1000 drugs as the toy dataset

In [7]:
# load AID1706 Assay Data
X_drugs, _, y = load_AID1706_SARS_CoV_3CL()

# we use the first 1000 molecules for this notebook
X_drugs = X_drugs[:1000]
y = y[:1000]

Beginning Processing...
100% [...............]Default binary threshold for the binding affinity scores is 15, recommended by the investigator
Done!


In [8]:
# We look at the first 10 SMILES strings
print (X_drugs[:10])

# We look at the first 10 labels
print (y[:10])

['CC1=C(SC(=N1)NC(=O)COC2=CC=CC=C2OC)C' 'CC1=CC=C(C=C1)C(=O)NCCCN2CCOCC2'
 'CSC1=CC=C(C=C1)C(=O)NC2CCSC3=CC=CC=C23'
 'CCOC(=O)N1CCC(CC1)N2CC34C=CC(O3)C(C4C2=O)C(=O)NC5=CC=C(C=C5)C'
 'CC1=CC(=NN1C(=O)C2=CC(=CC(=C2)[N+](=O)[O-])[N+](=O)[O-])C'
 'CC1=CC=C(C=C1)C(=O)CSC2=NN=C(N2CC3=CC=CO3)CNC4=C(C=C(C=C4)C)C'
 'CC(C1=CC(=C(C=C1)Cl)Cl)NC(=O)CCl'
 'CCOC(=O)CN1CC23C=CC(O2)C(C3C1=O)C(=O)NC4=CC5=C(C=C4)OCO5'
 'COC(=O)C1=CC=C(C=C1)COC(=O)C2=CC(=C(N=C2)Cl)Cl'
 'C1=CC=C2C(=C1)C=C(C(=O)O2)C3=C(C=C(C=C3)NC(=O)CC4=CC=C(C=C4)Cl)Cl']
[0 0 0 0 0 1 1 0 0 0]


In [9]:
# split the dataset into 80%: 20%

data = pd.DataFrame(np.stack([X_drugs, y]).T, columns = ["SMILES", "Label"])
train, test = data.iloc[:int(len(data)*0.8)], data.iloc[int(len(data)*0.8):]

In [10]:
print (train)

                                                SMILES Label
0                 CC1=C(SC(=N1)NC(=O)COC2=CC=CC=C2OC)C     0
1                      CC1=CC=C(C=C1)C(=O)NCCCN2CCOCC2     0
2               CSC1=CC=C(C=C1)C(=O)NC2CCSC3=CC=CC=C23     0
3    CCOC(=O)N1CCC(CC1)N2CC34C=CC(O3)C(C4C2=O)C(=O)...     0
4    CC1=CC(=NN1C(=O)C2=CC(=CC(=C2)[N+](=O)[O-])[N+...     0
..                                                 ...   ...
795  COC1=CC=CC=C1N(CC(=O)NC2=CC(=C(C=C2)Cl)C(=O)OC...     1
796                       COC1=C(C=C(C=C1Cl)C(=O)NN)Cl     1
797  C1CC(C2=CC=CC=C2C1)NC(=O)CCC(=O)N3CCN(CC3)S(=O...     0
798  CC(=O)NC1=CC=C(C=C1)N(C(C2=CC=C(C=C2)OC)C(=O)N...     1
799  CC(C)N(CC1=CC=CC=C1)CC(COC2=CC=CC3=C2C(=CN3)CC...     0

[800 rows x 2 columns]


In [11]:
print (test)

                                                SMILES Label
800             CC1=C(SC(=N1)N)C2=CSC(=N2)NC3=NC=CC=N3     0
801   C1=CC(=C(C=C1NC(=O)CN2C=C(N=C2)[N+](=O)[O-])Cl)F     0
802      C1C2CC3CC1CC(C2)(C3)C4=CC=C(C=C4)/C=N/NC(=S)N     1
803  COC1=CC=C(C=C1)NC(=S)N=NC2=C(N(C3=CC=CC=C32)CC...     0
804                   CC(C1=CC(=C(C=C1)Cl)Cl)NC(=O)CCl     1
..                                                 ...   ...
995  CC1=NC2=C(C(=NN2C(=C1)N3CCN(CC3)CC4=CC=CC=C4)C...     0
996  COC1=CC=CC(=C1)C(C(=O)NCC2=CC=CO2)N(CC3=CC=CO3...     1
997  COC1=CC(=CC(=C1OC)OC)/C=N\NC(=O)CN2C3=CC=CC=C3...     1
998     CN1C(=CC(=O)N(C1=O)C)NC(=O)C2=C(C=C(C=C2)Cl)Cl     0
999  CC(C(=O)NC(=O)NC)OC(=O)C1=C2CC/C(=C\C3=CC=CS3)...     0

[200 rows x 2 columns]


## 2. process the molecule SMILES strings into a graph structure (adjacency matrix)

In [12]:
from collections import defaultdict
import numpy as np
from rdkit import Chem
import torch


def create_atoms(mol, atom_dict):
    """Transform the atom types in a molecule (e.g., H, C, and O)
    into the indices (e.g., H=0, C=1, and O=2).
    Note that each atom index considers the aromaticity.
    """
    atoms = [a.GetSymbol() for a in mol.GetAtoms()]
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    atoms = [atom_dict[a] for a in atoms]
    return np.array(atoms)


def create_ijbonddict(mol, bond_dict):
    """Create a dictionary, in which each key is a node ID
    and each value is the tuples of its neighboring node
    and chemical bond (e.g., single and double) IDs.
    """
    i_jbond_dict = defaultdict(lambda: [])
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bond = bond_dict[str(b.GetBondType())]
        i_jbond_dict[i].append((j, bond))
        i_jbond_dict[j].append((i, bond))
    return i_jbond_dict


def extract_fingerprints(radius, atoms, i_jbond_dict,
                         fingerprint_dict, edge_dict):
    """Extract the fingerprints from a molecular graph
    based on Weisfeiler-Lehman algorithm.
    """

    if (len(atoms) == 1) or (radius == 0):
        nodes = [fingerprint_dict[a] for a in atoms]

    else:
        nodes = atoms
        i_jedge_dict = i_jbond_dict

        for _ in range(radius):

            """Update each node ID considering its neighboring nodes and edges.
            The updated node IDs are the fingerprint IDs.
            """
            nodes_ = []
            for i, j_edge in i_jedge_dict.items():
                neighbors = [(nodes[j], edge) for j, edge in j_edge]
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                nodes_.append(fingerprint_dict[fingerprint])

            """Also update each edge ID considering
            its two nodes on both sides.
            """
            i_jedge_dict_ = defaultdict(lambda: [])
            for i, j_edge in i_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    edge = edge_dict[(both_side, edge)]
                    i_jedge_dict_[i].append((j, edge))

            nodes = nodes_
            i_jedge_dict = i_jedge_dict_

    return np.array(nodes)

def create_dataset(data_in, radius=2):
    dataset = []

    for smiles, property in data_in.values:
        try:
            """Create each data with the above defined functions."""
            mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
            atoms = create_atoms(mol, atom_dict)
            molecular_size = len(atoms)
            i_jbond_dict = create_ijbonddict(mol, bond_dict)
            fingerprints = extract_fingerprints(radius, atoms, i_jbond_dict,
                                                fingerprint_dict, edge_dict)
            adjacency = Chem.GetAdjacencyMatrix(mol)
    
            """Transform the above each data of numpy
            to pytorch tensor on a device (i.e., CPU).
            """
            fingerprints = torch.LongTensor(fingerprints)
            adjacency = torch.FloatTensor(adjacency)
    
            dataset.append((fingerprints, adjacency, molecular_size, int(property)))
        except:
            pass

    return dataset

In [13]:
"""Initialize x_dict, in which each key is a symbol type
(e.g., atom and chemical bond) and each value is its index.
"""
atom_dict = defaultdict(lambda: len(atom_dict))
bond_dict = defaultdict(lambda: len(bond_dict))
fingerprint_dict = defaultdict(lambda: len(fingerprint_dict))
edge_dict = defaultdict(lambda: len(edge_dict))

dataset_train = create_dataset(train[["SMILES", "Label"]])
dataset_test = create_dataset(test[["SMILES", "Label"]])

N_fingerprints = len(fingerprint_dict)

In [14]:
# look at the first 3 data points in training
# they are at (fingerprints, adjacency, molecular_size, property) structure
print (dataset_train[:3])

[(tensor([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 30, 29, 28, 31,
        32, 33, 34, 34, 34, 35, 36, 36, 37, 37, 37, 37, 38, 38, 38, 34, 34, 34]), tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 36, 0), (tensor([46, 47, 48, 48, 49, 48, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 58,
        57, 34, 34, 34, 37, 37, 37, 37, 60, 61, 61, 62, 62, 61, 61, 61, 61, 36,
        36, 36, 36, 61, 61]), tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 41, 0), (tensor([68, 69, 70, 71, 48, 49, 48, 71, 50, 51, 72, 73, 74, 75, 76, 77, 71, 30,
        30, 48, 78, 79, 79, 79, 37, 37

## 3. Define the Molecule GNN model for molecule structure learning

In [18]:
class MolecularGNN(nn.Module):
    """
    borrowed largely from this repo https://github.com/masashitsubaki/molecularGNN_smiles
    """
    def __init__(self, N_fingerprints, dim, layer_gnn_hidden):
        super(MolecularGNN, self).__init__()
        self.embed_fingerprint = nn.Embedding(N_fingerprints, dim)
        self.W_fingerprint = nn.ModuleList([nn.Linear(dim, dim)
                                            for _ in range(layer_gnn_hidden)])
        self.W_property = nn.Linear(dim, 1)

    def pad(self, matrices, pad_value):
        """Pad the list of matrices
        with a pad_value (e.g., 0) for batch processing.
        For example, given a list of matrices [A, B, C],
        we obtain a new matrix [A00, 0B0, 00C],
        where 0 is the zero (i.e., pad value) matrix.
        """
        shapes = [m.shape for m in matrices]
        M, N = sum([s[0] for s in shapes]), sum([s[1] for s in shapes])
        zeros = torch.FloatTensor(np.zeros((M, N)))
        pad_matrices = pad_value + zeros
        i, j = 0, 0
        for k, matrix in enumerate(matrices):
            m, n = shapes[k]
            pad_matrices[i:i+m, j:j+n] = matrix
            i += m
            j += n
        return pad_matrices

    def update(self, matrix, vectors, layer):
        hidden_vectors = torch.relu(self.W_fingerprint[layer](vectors))
        return hidden_vectors + torch.matmul(matrix, hidden_vectors)

    def sum(self, vectors, axis):
        sum_vectors = [torch.sum(v, 0) for v in torch.split(vectors, axis)]
        return torch.stack(sum_vectors)

    def mean(self, vectors, axis):
        mean_vectors = [torch.mean(v, 0) for v in torch.split(vectors, axis)]
        return torch.stack(mean_vectors)

    def gnn(self, inputs):
        """Cat or pad each input data for batch processing."""
        fingerprints, adjacencies, molecular_sizes = inputs
        fingerprints = torch.cat(fingerprints)
        adjacencies = self.pad(adjacencies, 0)

        """GNN layer (update the fingerprint vectors)."""
        fingerprint_vectors = self.embed_fingerprint(fingerprints)
        for l in range(len(self.W_fingerprint)):
            hs = self.update(adjacencies, fingerprint_vectors, l)
            # fingerprint_vectors = F.normalize(hs, 2, 1)  # normalize.

        """Molecular vector by sum or mean of the fingerprint vectors."""
        molecular_vectors = self.sum(fingerprint_vectors, molecular_sizes)
        # molecular_vectors = self.mean(fingerprint_vectors, molecular_sizes)
        return molecular_vectors

    def forward(self, data_batch):
        molecular_vectors = self.gnn(data_batch)
        predicted_scores = self.W_property(molecular_vectors)
        return predicted_scores

model = MolecularGNN(N_fingerprints, 128, 2)

## 4. training the model for 10 epochs

In [19]:
from sklearn.metrics import roc_auc_score

batch_size = 32
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

for epoch in range(15):
    """ model training """
    np.random.shuffle(dataset_train)
    train_loss = 0

    model.train()
    for i in range(0, len(dataset_train), batch_size):
        data_batch = list(zip(*dataset_train[i: i+batch_size]))
        
        # feed features into the model
        pred = model.forward(data_batch[:3]).squeeze(-1)
        
        # use gt property as labels
        label = torch.FloatTensor(data_batch[-1])
        loss = criterion(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    """ model evaluation """
    model.eval()

    predicted, groundtruth = [], []
    with torch.no_grad():
        for i in range(0, len(dataset_test), batch_size):
            data_batch = list(zip(*dataset_train[i: i+batch_size]))
            # feed features into the model
            pred = model.forward(data_batch[:3]).squeeze(-1).numpy().tolist()
            label = data_batch[-1]

            predicted += pred
            groundtruth += label
        
    print (f"--- epoch: {epoch} ---, train loss: {train_loss}, test AUROC: {roc_auc_score(groundtruth, predicted)}")

--- epoch: 0 ---, train loss: 39.32982802391052, test AUROC: 0.5515151515151515
--- epoch: 1 ---, train loss: 24.156920433044434, test AUROC: 0.6504513140027158
--- epoch: 2 ---, train loss: 18.350540339946747, test AUROC: 0.7955907021327583
--- epoch: 3 ---, train loss: 15.509063512086868, test AUROC: 0.8324640937174036
--- epoch: 4 ---, train loss: 12.810607969760895, test AUROC: 0.8664265706282513
--- epoch: 5 ---, train loss: 11.102667361497879, test AUROC: 0.9002656363197294
--- epoch: 6 ---, train loss: 9.69138415157795, test AUROC: 0.9347860791826309
--- epoch: 7 ---, train loss: 8.683858707547188, test AUROC: 0.9528820856254485
--- epoch: 8 ---, train loss: 7.6352120488882065, test AUROC: 0.9483418367346939
--- epoch: 9 ---, train loss: 6.850203737616539, test AUROC: 0.969544766004943
--- epoch: 10 ---, train loss: 6.049233138561249, test AUROC: 0.9728867623604466
--- epoch: 11 ---, train loss: 5.406656213104725, test AUROC: 0.989516129032258
--- epoch: 12 ---, train loss: 4.94